In [2]:
import os
import gzip
import json
import numpy as np
import pandas as pd
import tqdm
pd.set_option('display.max_rows', 100)

## Heroes information and stats

In [6]:
with open("../data/heroes.json", "r") as fp: 
    heroes = json.load(fp)
    
df_heroes = pd.DataFrame(heroes)

In [4]:
df_heroes_dict = df_heroes[["id", "localized_name"]]
df_heroes_dict

,id,localized_name
0,1,Anti-Mage
1,2,Axe
2,3,Bane
3,4,Bloodseeker
4,5,Crystal Maiden
...,...,...
117,126,Void Spirit
118,128,Snapfire
119,129,Mars
120,135,Dawnbreaker


As seen in the table above, the heroids in dota are not incremental, but have gaps. We have to make a `id to heroid` map and vice versa for lookup.

### Notes:

* We will look from the alliance side
* There are 121 heroes as of 7.30 patch
* Newest hero is Dawnbreaker
* 63 ranged, 58 melee
* 43 int, 40 str, 38 str

## Preprocessing loop

In [8]:
def get_comp_by_training_sample(sample):
    radiant_win = sample[-1]
    radiant = []
    dire = []
    for i, h in enumerate(sample[:-1]):
        if h == 1:
            radiant.append(_get_hero_name_by_id(i))
        elif h == -1:
            dire.append(_get_hero_name_by_id(i))
    print(radiant, dire, radiant_win)

def _get_hero_name_by_id(idx: int):
    return df_heroes[df_heroes["id"].index == idx]["localized_name"].values[0]

In [5]:
id_to_hid = df_heroes[["id"]].to_dict()["id"]
hid_to_id = {v:k for k,v in id_to_hid.items()}

results = []
seen_matches = set()
dup_counter = 0
invalid = 0
# [[1, 0, -1, ... , n_heroes_121, winner], ... ]
count = 0
with gzip.open("../data/raw/matches_5148330922-5148330922.gz", "r") as fp:
    for line in tqdm.tqdm(fp):
        training_sample = np.zeros((122,))
        match = json.loads(line)
        players = match["players"]
        match_id = match["match_id"]
        invalid_index = False
        
        if not "match_id" in match:
            # invalid += 1 
            continue
        
        if len(players) != 10:
            # invalid += 1
            continue
        
        if match_id in seen_matches:
            # dup_counter += 1
            continue
        
        

        for p in players:
            if p["hero_id"] not in hid_to_id.keys():
                invalid_index = True
                # invalid += 1
                break

            hero_i = hid_to_id[p["hero_id"]]

            is_radiant = True if p["player_slot"] <= 7 else False # DETTE ER RIKTIG!!! 

            if is_radiant: # radiant slot
                training_sample[hero_i] = 1
            else: 
                training_sample[hero_i] = -1
        
        if invalid_index:
            continue

        training_sample[-1] = match["radiant_win"]
        results.append(training_sample)
        seen_matches.add(match_id)
        break
        
# print("# duplicates", dup_counter)
print("seen matches: {}, len samples: {}".format(len(seen_matches), len(results)))
# print("# invalid", invalid)

# with open("../data/preprocessed/test_dataset.npy", "wb") as fp:
#     np.save(fp, np.array(results, dtype=np.int8), allow_pickle=True)

0it [00:00, ?it/s]

seen matches: 1, len samples: 1


In [28]:
pp_matches = np.load("../data/matches_5148330922-5148330922.npy")

print(len(pp_matches))

# [h1_carry, h1_mid, h1_off, h1_jungle, h1_support, h2_carry, h2_mid, h2_off, h2_jungle, h2_support]



401
